In [1]:
# Author Davide Aloi
# Scripts for the analyses of the EEG data (active task) collected at the Wellington Hospital - Aloi Davide PhD UoB

In [2]:
%matplotlib
import numpy as np
import mne
import os
import glob 
import cupy
import matplotlib.pyplot as plt

mne.set_log_level("WARNING")
mne.cuda.init_cuda(verbose=True)
mne.utils.set_config('MNE_USE_CUDA', 'true')  

# Folders to the EEG active task data
output_folder = 'D:\\Raindrop_eeg_analysis\\p01\\'
sessions = ['D:\\Raindrop_data\\p01\\p01_w01\\eeg_baseline\\active_task\\', # baseline w 1
            'D:\\Raindrop_data\\p01\\p01_w02\\day04_eeg\\active_task\\', # post w 2
            'D:\\Raindrop_data\\p01\\p01_w03\\eeg_baseline\\active_task\\', # baseline w 3
            'D:\\Raindrop_data\\p01\\p01_w04\\day04_eeg\\active_task\\', # post w 4
            'D:\\Raindrop_data\\p01\\p01_w05\\eeg_baseline\\active_task\\', # baseline w5
            'D:\\Raindrop_data\\p01\\p01_w06\\day04_eeg\\active_task\\'] # post w6

# Paths to .mff files
sessions_raw = []
for n, session in enumerate(sessions):
    sessions_raw.append(glob.glob(session + '\*.mff')[0])

raw = mne.io.read_raw_egi(sessions_raw[0], preload=True)

Using matplotlib backend: <object object at 0x000002BD343362F0>
Now using CUDA device 0
Enabling CUDA with 3.28 GB available memory


In [3]:
montage = mne.channels.make_standard_montage('GSN-HydroCel-128')
raw_filtered = raw.copy().filter(1, 40)
picks_eeg = mne.pick_types(raw.info, meg=False, eeg=True, eog=True, stim=False,
                           exclude='bads')
events = mne.find_events(raw, stim_channel='STI 014', verbose=True) # or STIM
montage = mne.channels.make_standard_montage('GSN-HydroCel-128')
#montage.plot()  # 2D
#fig = montage.plot(kind='3d', show=False)  # 3D
#fig = fig.gca().view_init(azim=70, elev=15)  # set view angle for tutorial

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


72 events found
Event IDs: [1 2]


In [ ]:
event_id = dict(move=1,relax=2)
#event_id = dict(move=1)

# epoching ALL DATA without baseline correction for bad channel detection
epochs = mne.Epochs(raw_filtered, events, event_id, 0, 2,
                baseline=None, reject=None,
                verbose=False, detrend=0, preload=True)

# RANSAC bad channel detection
from autoreject import Ransac  # this uses ransac from PREP
ransac = Ransac(verbose=True, picks=picks_eeg, n_jobs=1)
epochs_clean = ransac.fit_transform(epochs)
bad_chs = ransac.bad_chs_ # list with bad channels according to RANSAC 

# Workflow: autoreject -> ica -> autoreject
# The workflow is suggested here 
# https://autoreject.github.io/stable/auto_examples/plot_autoreject_workflow.html#sphx-glr-auto-examples-plot-autoreject-workflow-py

# Bad epochs detection (BEFORE ICA)
import autoreject
ar = autoreject.AutoReject(n_interpolate=[1, 2, 3, 4], random_state=11,
                           n_jobs=1, verbose=True)
ar.fit(epochs[:20])  # fit on a few epochs to save time
epochs_ar, reject_log = ar.transform(epochs, return_log=True)
reject_log.plot('horizontal')
print(f'Rejected Epochs: {(reject_log.bad_epochs == True).sum()}')
# reject_log.bad_epochs

# ICA without bad epochs / bad channels
epochs.info['bads'] = bad_chs
ica = mne.preprocessing.ICA(random_state=99)
ica.fit(epochs[~reject_log.bad_epochs])

# here you need to plot ICA and exclude bad components by clicking on them
ica.plot_components(picks = np.arange(0,30,1))
print(ica.exclude) # list of components excluded
ica.plot_components(ica.exclude)
ica.plot_overlay(epochs.average(), exclude=ica.exclude)
ica.apply(epochs, exclude=ica.exclude)

# Autoreject after high pass filter and ICA
epochs.info['bads'] = []
ar = autoreject.AutoReject(n_interpolate=[1, 2, 3, 4], random_state=11,
                           n_jobs=1, verbose=True)

ar.fit(epochs[:20])  # fit on the first 20 epochs to save time
epochs_ar, reject_log = ar.transform(epochs, return_log=True)
epochs[reject_log.bad_epochs].plot(scalings=dict(eeg=100e-6))
reject_log.plot('horizontal')

evoked_bad = epochs[reject_log.bad_epochs].average()
plt.figure()
plt.plot(evoked_bad.times, evoked_bad.data.T * 1e6, 'r', zorder=-1)
epochs_ar.average().plot(axes=plt.gca())

mne.set_eeg_reference(epochs_ar, ref_channels='average', copy = False)